In [1]:
# Libraries needed for NLP
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

# Libraries needed for Tensorflow processing
import tensorflow as tf
import numpy as np
import random
import json

[nltk_data] Downloading package punkt to C:\Users\THE
[nltk_data]     FSR\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import json
# import our chat-bot intents file
with open('intentos.json') as json_data:
    intents = json.load(json_data)

In [3]:
intents

{'intents': [{'tag': 'saludos',
   'patterns': ['Hola',
    'Como estas',
    'Hay alguien aqui?',
    'Hello',
    'Buenos dias',
    'Saludos',
    'Que tal',
    'Como te va',
    'Hola, que tal?'],
   'responses': ['Hola, gracias por tu visita',
    'Gusto en verte de nuevo',
    'Hola, en que puedo ayudarte?',
    'Hola! Como estas?',
    'Buenos dias! En que puedo ayudarte?'],
   'context_set': ''},
  {'tag': 'despedidas',
   'patterns': ['adios',
    'nos vemos luego',
    'regresa pronto',
    'hasta luego',
    'chau',
    'me tengo que ir'],
   'responses': ['Hasta luego, gracias por tu visita',
    'Que tengas un buen dia',
    'Adios! Vuelve pronto',
    'Cuidate, hasta la proxima!',
    'Nos vemos pronto, adios!']},
  {'tag': 'agradecimientos',
   'patterns': ['gracias',
    'muchas gracias',
    'te lo agradezco',
    'eso es de ayuda',
    'thank you',
    'thanks'],
   'responses': ['De nada!',
    'A ti!',
    'Con gusto!',
    'Feliz de ayudarte!',
    'Para eso estoy

In [4]:
words = []
classes = []
documents = []
ignore = ['?']
# loop through each sentence in the intent's patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each and every word in the sentence
        w = nltk.word_tokenize(pattern)
        # add word to the words list
        words.extend(w)
        # add word(s) to documents
        documents.append((w, intent['tag']))
        # add tags to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [6]:
# Perform stemming and lower each word as well as remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

# remove duplicate classes
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

37 documents
7 classes ['acerca_de', 'agradecimientos', 'chatbot', 'despedidas', 'peliculas', 'saludos', 'ubicacion']
63 unique stemmed words [',', 'adio', 'agradezco', 'alguien', 'alguna', 'aqui', 'ayuda', 'bueno', 'chatbot', 'chau', 'como', 'construyo', 'cual', 'cuentam', 'de', 'del', 'dia', 'direccion', 'dond', 'encontrart', 'ere', 'es', 'eso', 'est', 'esta', 'esto', 'favorita', 'gracia', 'hablam', 'hasta', 'hay', 'hello', 'hola', 'ir', 'llama', 'lo', 'luego', 'me', 'mucha', 'no', 'pelicula', 'pronto', 'puedo', 'que', 'quien', 'recomienda', 'recomiendam', 'regresa', 'saludo', 'se', 'sobr', 'sugerem', 'tal', 'te', 'tengo', 'thank', 'ti', 'tu', 'ubicacion', 'ubicado', 'va', 'vemo', 'you']


In [7]:
import numpy as np
import random

# Suponiendo que 'documents', 'classes', 'words' y 'stemmer' ya están definidos

# create training data
training = []
output = []
# create an empty array for output
output_empty = [0] * len(classes)

# create training set, bag of words for each sentence
for doc in documents:
    # initialize bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stemming each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is '1' for current tag and '0' for rest of other tags
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# Verificar que todas las listas internas tengan la misma longitud
bag_lengths = [len(t[0]) for t in training]
output_lengths = [len(t[1]) for t in training]

if len(set(bag_lengths)) != 1 or len(set(output_lengths)) != 1:
    raise ValueError("Inconsistent lengths in training data")

# shuffling features and turning it into np.array
random.shuffle(training)
training = np.array(training, dtype=object)

# creating training lists
train_x = np.array([t[0] for t in training])
train_y = np.array([t[1] for t in training])


In [8]:
import tensorflow as tf
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10,input_shape=(len(train_x[0]),)))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Dense(len(train_y[0]), activation='softmax'))
model.compile(tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy']) 

c:\Users\THE FSR\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
model.fit(np.array(train_x), np.array(train_y), epochs=1000, batch_size=8, verbose=1)
model.save("model.keras")

Epoch 1/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0354 - loss: 2.0703   
Epoch 2/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0666 - loss: 2.0286 
Epoch 3/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1190 - loss: 1.9868 
Epoch 4/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1072 - loss: 1.9909 
Epoch 5/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0933 - loss: 1.9732     
Epoch 6/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0919 - loss: 1.9882     
Epoch 7/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1301 - loss: 1.9341     
Epoch 8/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2061 - loss: 1.9277 
Epoch 9/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2616 - loss: 1.8724 
Epoch 10/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2883 - loss: 1.8819 
Epoch 11/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3147 - loss: 1.8279 
Epoch 12/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/ste

In [10]:
import pickle
pickle.dump( {'words':words, 'classes':classes}, open( "training_data", "wb" ) )

In [11]:
from keras.models import load_model
model = load_model("model.keras")

In [12]:
# restoring all the data structures
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']

In [13]:
with open('intentos.json') as json_data:
    intents = json.load(json_data)

In [14]:
def clean_up_sentence(sentence):
    # tokenizing the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stemming each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# returning bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words):
    # tokenizing the pattern
    sentence_words = clean_up_sentence(sentence)
    # generating bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
    bag=np.array(bag)
    return(bag)

In [20]:
ERROR_THRESHOLD = 0.30
def classify(sentence):
    # generate probabilities from the model
    bag = bow(sentence, words)
    results = model.predict(np.array([bag]))
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results[0]) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # a random response from the intent
                    return print(random.choice(i['responses']))

            results.pop(0)

In [21]:

print(response("Gracias"))
print(response("adios"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Para eso estoy!
None
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Que tengas un buen dia
None


In [22]:
print(response("Hola"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Hola! Como estas?
None


In [23]:
response('Bye')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Hola, en que puedo ayudarte?


In [24]:
response("Quien eres?")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Hola, somos MasterBot. Encantado de conocerte. Creamos este chatbot para proyecto final de nuestro curso de inteligencia artificial.
